In [1]:
import pandas as pd
from roboto.models import Candle, Instrument
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, LSTM, Reshape, Lambda, RepeatVector
from xgboost import XGBClassifier

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
instruments = Instrument.objects.all()

In [5]:
frames = []

for instrument in instruments:
    columns = ('close',)
    candles = list(instrument.candles.values_list('time', *columns))
    dataframe_columns = (['{}_{}'.format(instrument.name, c) for c in columns])
    tmp_df = pd.DataFrame(
        columns=['time', *dataframe_columns],
        data=candles,
    )
    tmp_df.set_index(['time'], inplace=True)
    frames.append(tmp_df)

In [6]:
data = pd.concat(frames, axis=1, ignore_index=False)

In [7]:
# plt.figure(figsize(20,2))
# for column in data.columns:
#     print (column)
#     data[column].plot()
#     plt.show()

In [8]:
first_full_field_index = None
for index, row in data.iterrows():
    if not row.isna().any():
        first_full_field_index = index
        break
first_full_field_index

Timestamp('2018-08-15 07:00:00+0000', tz='UTC')

In [9]:
data = data[first_full_field_index:]

In [10]:
data = data.fillna(method='ffill')

In [11]:
data.isna().any().any()

False

In [3]:
def create_df_with_diff(df, T):
    m, f = df.shape
    diff_df = None
    for t in range(1, T+1):
        tmp_v = df.diff(t).values.reshape(m,1,f)
        if diff_df is None:
            diff_df = tmp_v
        else:
            diff_df = np.hstack([diff_df, tmp_v])
    return diff_df[T:]

In [4]:
scaler = StandardScaler()
# scaled_data = scaler.fit_transform(data)
scaled_data = data

NameError: name 'data' is not defined

In [5]:
new_data = create_df_with_diff(scaled_data, 5)

NameError: name 'scaled_data' is not defined

In [15]:
def reshape_to_2dim(df):
    return df.reshape(df.shape[0], df.shape[1] * df.shape[2])

In [16]:
# for target_i in range(new_data.shape[2]):
#     print (target_i, end='--------\n')
    
#     Y = (new_data[1:,0,target_i] > 0) * 1
#     X = reshape_to_2dim(new_data[:-1])
    
#     model 

In [17]:
model_history = {}

In [18]:
for target_i in range(new_data.shape[2])[100:]:
    target_column = data.columns[target_i]
    print (target_column, end='\n--------\n\n')
    
    Y = (new_data[1:,0,target_i] > 0) * 1
    X = new_data[:-1]
    T = 20
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=False)
    
    model = Sequential()
    model.add(LSTM(20, input_shape=X.shape[1:]))
    model.add(Dropout(0.3))
    model.add(Flatten(input_shape=X.shape[1:]))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    optimizer = optimizers.Adam(lr=0.001,)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    model.fit(
        X_train,
        Y_train,
        epochs=1,
        batch_size=128,
        validation_data=(X_test, Y_test),
        verbose=2,
        shuffle=False,
    )
    
    model_history[target_column] = model

XAU_AUD_close
--------

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.7423 - acc: 0.4822 - val_loss: 0.6999 - val_acc: 0.5112
USB02Y_USD_close
--------

Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.7717 - acc: 0.5296 - val_loss: 0.7002 - val_acc: 0.5490
EUR_HKD_close
--------

Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.8079 - acc: 0.5025 - val_loss: 0.7279 - val_acc: 0.4839
XAU_JPY_close
--------

Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.7800 - acc: 0.5009 - val_loss: 0.7144 - val_acc: 0.4952
TWIX_USD_close
--------

Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.8414 - acc: 0.4600 - val_loss: 0.7273 - val_acc: 0.4864
EUR_JPY_close
--------

Train on 3737 samples, validate on 1246 samples
 - 1s - loss: 0.7

In [19]:
class ModelHisotry(object):
    model_values = ['loss', 'acc', 'val_loss', 'val_acc']
    
    def __init__(self, name, history_obj):
        self.name = name
        for key in self.model_values:
            setattr(self, key, history_obj[key][-1])
            
    def __repr__(self):
        f = {}
        for key in self.model_values:
            f[key] = [getattr(self, key)]
        return str(f)
    
                
    def __str__(self):
        return '{}: {}'.format(self.name, self.__repr__())
    
    def to_csv(self, sep=', '):
        return sep.join([str(getattr(self, key)) for key in ['name'] + self.model_values])
    
    @classmethod
    def from_csv(cls, value_string, sep=', '):
        h_obj = {}
        values = value_string.split(sep)
        for value, key in zip(values, ['name'] + cls.model_values):
            h_obj[key] = [value]
        
        name = h_obj.pop('name')[0]
        
        return cls(name, h_obj)

In [20]:
for key, model in model_history.items():
    h = ModelHisotry(key, model.history.history)
    print (h.to_csv())

XAU_AUD_close, 0.7422574997458637, 0.48220497, 0.6999254077433774, 0.51123595
USB02Y_USD_close, 0.7716926300401257, 0.52956915, 0.7001667488635447, 0.54895663
EUR_HKD_close, 0.807896382259372, 0.50254214, 0.727900325008227, 0.48394865
XAU_JPY_close, 0.779987217718374, 0.50093657, 0.7143730586260126, 0.4951846
TWIX_USD_close, 0.8414444474571683, 0.45999464, 0.7272717771905192, 0.48635635
EUR_JPY_close, 0.7403982835304683, 0.48889485, 0.709941552309125, 0.48154092
WHEAT_USD_close, 0.6947274222902222, 0.6023548, 0.6676901182814547, 0.62760836
GBP_PLN_close, 0.7509782711930667, 0.49157077, 0.7141105458977517, 0.49036917
GBP_JPY_close, 0.7239843322562763, 0.50789404, 0.70035291368659, 0.4975923
USD_TRY_close, 0.8811321022008427, 0.49906343, 0.7497135192203674, 0.5144462
HK33_HKD_close, 0.776050368381242, 0.52609044, 0.6622669117025923, 0.62760836
EUR_CAD_close, 0.7775969693518463, 0.49344394, 0.7228372443353767, 0.4670947
USD_SEK_close, 0.7567494117642648, 0.4961199, 0.7089964781871385, 0.5

In [21]:
fname = 'shallow_train_results.csv'

In [28]:
hist = []

with open('shallow_train_results.csv', 'r') as f:
    for line in f:
        h = ModelHisotry.from_csv(line)
        hist.append(h)

In [33]:
sorted_hist = sorted(hist, key=lambda x: x.acc, reverse=True)

In [38]:
for h in sorted_hist:
    print ('{}: {}'.format(h.name, h.acc))

USD_SAR_close: 0.7390955
UK10YB_GBP_close: 0.66657746
EU50_EUR_close: 0.6620284
SUGAR_USD_close: 0.6304522
WHEAT_USD_close: 0.6023548
USB10Y_USD_close: 0.5721167
DE30_EUR_close: 0.5520471
UK100_GBP_close: 0.5512443
DE10YB_EUR_close: 0.5341183
USB02Y_USD_close: 0.52956915
HK33_HKD_close: 0.52609044
SOYBN_USD_close: 0.5242173
NATGAS_USD_close: 0.5228793
FR40_EUR_close: 0.5199358
USD_INR_close: 0.5185978
NZD_JPY_close: 0.5159219
GBP_CAD_close: 0.5153867
EUR_CZK_close: 0.51378113
ZAR_JPY_close: 0.51378113
XAG_USD_close: 0.5135135
XAG_CAD_close: 0.5124431
CAD_HKD_close: 0.51190794
NZD_CHF_close: 0.51137275
XAU_SGD_close: 0.51137275
USB05Y_USD_close: 0.51137275
CAD_CHF_close: 0.51030236
EUR_GBP_close: 0.51030236
USD_THB_close: 0.5097672
XAU_USD_close: 0.509232
GBP_CHF_close: 0.5081616
GBP_JPY_close: 0.50789404
USD_HKD_close: 0.50682366
XAG_EUR_close: 0.50628847
NZD_CAD_close: 0.50628847
XAU_EUR_close: 0.5054857
EUR_NOK_close: 0.5052181
USD_ZAR_close: 0.5052181
XAU_GBP_close: 0.5049505
CAD_JP